In [2]:
import requests
import pandas as pd
import numpy as np
import pyodbc
from sqlalchemy import create_engine
#import pymysql
import time
import urllib
import datetime

In [33]:
### a list of the interested company

stock_list = ["AAPL","MSFT"]
stock_test = ["GICS","CIK"]
TICKERS_SP500 = ("GICS,CIK,MMM,ABT,ABBV,ACN,ACE,ACT,ADBE,ADT,AES,AET,AFL," +
                "AMG,A,GAS,APD,ARG,AKAM,AA,ALXN,ATI,ALLE,ADS,ALL,ALTR,MO," + 
                "AMZN,AEE,AAL,AEP,AXP,AIG,AMT,AMP,ABC,AME,AMGN,APH,APC,ADI," + 
                "AON,APA,AIV,AAPL,AMAT,ADM,AIZ,T,ADSK,ADP,AN,AZO,AVGO,AVB,AVY," + 
                "BHI,BLL,BAC,BK,BCR,BAX,BBT,BDX,BBBY,BBY,BIIB,BLK,HRB,BA,BWA," + 
                "BXP,BSX,BMY,BRCM,BFB,CHRW,CA,CVC,COG,CAM,CPB,COF,CAH,HSIC,KMX," + 
                "CCL,CAT,CBG,CBS,CELG,CNP,CTL,CERN,CF,SCHW,CHK,CVX,CMG,CB,CI,XEC," + 
                "CINF,CTAS,CSCO,C,CTXS,CLX,CME,CMS,COH,KO,CCE,CTSH,CL,CMA,CSC,CAG,COP,CNX,ED,STZ,GLW,COST,CCI,CSX,CMI,CVS,DHI,DHR,DRI,DVA,DE,DLPH,DAL,XRAY,DVN,DO,DTV,DFS,DG,DLTR,D,DOV,DOW,DPS,DTE,DD,DUK,DNB,ETFC,EMN,ETN,EBAY,ECL,EIX,EW,EA,EMC,EMR,ENDP,ESV,ETR,EOG,EQT,EFX,EQIX,EQR,ESS,EL,ES,EXC,EXPE,EXPD,ESRX,XOM,FFIV,FB,FDO,FAST,FDX,FIS,FITB,FSLR,FE,FISV,FLIR,FLS,FLR,FMC,FTI,F,FOSL,BEN,FCX,FTR,GME,GCI,GPS,GRMN,GD,GE,GGP,GIS,GM,GPC,GNW,GILD,GS,GT,GOOG,GWW,HAL,HBI,HOG,HAR,HRS,HIG,HAS,HCA,HCP,HCN,HP,HES,HPQ,HD,HON,HRL,HSP,HST,HCBK,HUM,HBAN,ITW,IR,TEG,INTC,ICE,IBM,IP,IPG,IFF,INTU,ISRG,IVZ,IRM,JEC,JNJ,JCI,JOY,JPM,JNPR,KSU,K,KEY,GMCR,KMB,KIM,KMI,KLAC,KSS,KRFT,KR,LB,LLL,LH,LRCX,LM,LEG,LEN,LVLT,LUK,LLY,LNC,LLTC,LMT,L,LO,LOW,LYB,MTB,MAC,M,MNK,MRO,MPC,MAR,MMC,MLM,MAS,MA,MAT,MKC,MCD,MHFI,MCK,MJN,MWV,MDT,MRK,MET,KORS,MCHP,MU,MSFT,MHK,TAP,MDLZ,MON,MNST,MCO,MS,MOS,MSI,MUR,MYL,NDAQ,NOV,NAVI,NTAP,NFLX,NWL,NFX,NEM,NWSA,NEE,NLSN,NKE,NI,NE,NBL,JWN,NSC,NTRS,NOC,NRG,NUE,NVDA,ORLY,OXY,OMC,OKE,ORCL,OI,PCAR,PLL,PH,PDCO,PAYX,PNR,PBCT,POM,PEP,PKI,PRGO,PFE,PCG,PM,PSX,PNW,PXD,PBI,PCL,PNC,RL,PPG,PPL,PX,PCP,PCLN,PFG,PG,PGR,PLD,PRU,PEG,PSA,PHM,PVH,QEP,PWR,QCOM,DGX,RRC,RTN,RHT,REGN,RF,RSG,RAI,RHI,ROK,COL,ROP,ROST,RCL,R,CRM,SNDK,SCG,SLB,SNI,STX,SEE,SRE,SHW,SIAL,SPG,SWKS,SLG,SJM,SNA,SO,LUV,SWN,SE,STJ,SWK,SPLS,SBUX,HOT,STT,SRCL,SYK,STI,SYMC,SYY,TROW,TGT,TEL,TE,THC,TDC,TSO,TXN,TXT,HSY,TRV,TMO,TIF,TWX,TWC,TJX,TMK,TSS,TSCO,RIG,TRIP,FOXA,TSN,TYC,USB,UA,UNP,UNH,UPS,URI,UTX,UHS,UNM,URBN,VFC,VLO,VAR,VTR,VRSN,VZ,VRTX,VIAB,V,VNO,VMC,WMT,WBA,DIS,WM,WAT,ANTM,WFC,WDC,WU,WY,WHR,WFM,WMB,WIN,WEC,WYN,WYNN,XEL,XRX,XLNX,XL,XYL,YHOO,YUM,ZMH,ZION,ZTS,SAIC,AP")

Full_Stock_list = TICKERS_SP500.split(',')

In [5]:
def connect_to_engine():
    '''
    Connect to Azure SQL database using my credential
    '''
    params = urllib.parse.quote_plus(r'Driver={ODBC Driver 17 for SQL Server};Server=tcp:danpersonalproject.database.windows.net,1433;Database=StockMarket;Uid=danvib@danpersonalproject;Pwd=Dan024721418!;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')
    conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
    engine_azure = create_engine(conn_str,echo=True)
    return engine_azure

In [7]:
def pull_Full_stock_data(Stock_Names,engine):
    '''
    Take in Stock List and specified engine
    ETL process
    E: Extract data from NASDAQ
    T: use panda dataframe to transform data into desired format
    L: use SQL sqlalchemy to 
            PULL
                Closing pricing
                date
                highest trade price
                low Price
                Open Price
                Trade Volumn
                Stock Name
    '''
    for name in Stock_Names:
        r = requests.get(
            "https://api.nasdaq.com/api/quote/"+ name + "/historical?assetclass=stocks&fromdate=2014-12-09&limit=1500&todate=2019-12-19").json()
        if r["data"] != None:
            df = pd.DataFrame(r["data"]["tradesTable"]["rows"])
            df["close"] = df["close"].str.replace('$','')
            df["high"] = df["high"].str.replace('$','')
            df["low"] = df["low"].str.replace('$','')
            df["open"] = df["open"].str.replace('$','')
            df["volume"] = df["volume"].str.replace(',','')
            df['Stock'] = name
            df.to_sql('Raw.Stock_Transaction_NASDAQ',con=engine, if_exists='append')


In [24]:
def pull_company_info(Stock_Names,engine):
    for name in Stock_Names:
        r = requests.get(
            'https://api.nasdaq.com/api/quote/'+name+'/summary?assetclass=stocks').json()
        if r["data"] != None:
            df = pd.DataFrame(r['data']['summaryData'])
            df.drop('label',inplace=True)
            df.reset_index(inplace=True)
            df[['FiftTwo Week High','FiftTwo Week Low']] = df['FiftTwoWeekHighLow'].apply( lambda x: pd.Series(str(x).split("/"))) 
            df.drop(['index','TodayHighLow','PreviousClose','FiftTwoWeekHighLow'], axis =1, inplace=True)

            for item in df.columns:
                if item not in ['PERatio','Beta']:
                    df[item] =  df[item].str.replace('$','')
                    df[item] =  df[item].str.replace(',','')
                    df[item] =  df[item].str.replace('%','')
                    
            ##### need to cast data type to VARCHAR(MAX) when creating these tables        
            if 'SpecialDividendDate' not in df.columns:
                df['SpecialDividendDate'] = np.nan
                
            if 'SpecialDividendAmount' not in df.columns:
                df['SpecialDividendAmount'] = np.nan

            if 'SpecialDividendPaymentDate' not in df.columns:
                df['SpecialDividendPaymentDate'] = np.nan
                
            df['Stock'] = name
            df.to_sql('Raw.Stage_Stock_info',con=engine, if_exists='append')
            #time.sleep(2)

In [25]:
engine = connect_to_engine()
Stock_Names = TICKERS_SP500.split(',')

In [32]:
#pull_Full_stock_data(Stock_Names,engine)
pull_company_info(Stock_Names,engine)

2019-12-26 16:18:08,556 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA], [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_NAME], [INFORMATION_SCHEMA].[COLUMNS].[IS_NULLABLE], [INFORMATION_SCHEMA].[COLUMNS].[DATA_TYPE], [INFORMATION_SCHEMA].[COLUMNS].[ORDINAL_POSITION], [INFORMATION_SCHEMA].[COLUMNS].[CHARACTER_MAXIMUM_LENGTH], [INFORMATION_SCHEMA].[COLUMNS].[NUMERIC_PRECISION], [INFORMATION_SCHEMA].[COLUMNS].[NUMERIC_SCALE], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_DEFAULT], [INFORMATION_SCHEMA].[COLUMNS].[COLLATION_NAME] 
FROM [INFORMATION_SCHEMA].[COLUMNS] 
WHERE [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2019-12-26 16:18:08,557 INFO sqlalchemy.engine.base.Engine ('Raw.Stage_Stock_info', 'dbo')
2019-12-26 16:18:08,633 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-26 16:18:08,636 INFO sqlalchemy.

2019-12-26 16:18:10,073 INFO sqlalchemy.engine.base.Engine (0, 'NASDAQ-GS', 'Technology', 'Computer Software: Prepackaged Software', '105.00', '505903', '1649680', '14488259621', 23.82, '28.75', '4.67', '1.40', 'Dec 5 2019', 'Dec 20 2019', '1.26499', 0.85, '114.73', '90.28', None, None, None, 'CTXS')
2019-12-26 16:18:10,138 INFO sqlalchemy.engine.base.Engine COMMIT
2019-12-26 16:18:10,164 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) ORDER BY [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME]
2019-12-26 16:18:10,165 INFO sqlalchemy.engine.base.Engine ('dbo', 'BASE TABLE')
2019-12-26 16:18:10,530 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA], [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_NAME

2019-12-26 16:18:12,195 INFO sqlalchemy.engine.base.Engine ('Raw.Stage_Stock_info', 'dbo')
2019-12-26 16:18:12,264 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-26 16:18:12,266 INFO sqlalchemy.engine.base.Engine INSERT INTO [Raw.Stage_Stock_info] ([index], [Exchange], [Sector], [Industry], [OneYrTarget], [ShareVolume], [AverageVolume], [MarketCap], [PERatio], [ForwardPE1Yr], [EarningsPerShare], [AnnualizedDividend], [ExDividendDate], [DividendPaymentDate], [Yield], [Beta], [FiftTwo Week High], [FiftTwo Week Low], [SpecialDividendDate], [SpecialDividendAmount], [SpecialDividendPaymentDate], [Stock]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2019-12-26 16:18:12,267 INFO sqlalchemy.engine.base.Engine (0, 'NYSE', 'Consumer Non-Durables', 'Beverages (Production/Distribution)', '60.00', '6062461', '11296616', '235732715563', 30.57, '26.05', '1.80', '1.60', 'Nov 29 2019', 'Dec 16 2019', '2.92449', 0.48, '55.92', '44.42', None, None, None, 'KO')
20

2019-12-26 16:18:14,645 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA], [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_NAME], [INFORMATION_SCHEMA].[COLUMNS].[IS_NULLABLE], [INFORMATION_SCHEMA].[COLUMNS].[DATA_TYPE], [INFORMATION_SCHEMA].[COLUMNS].[ORDINAL_POSITION], [INFORMATION_SCHEMA].[COLUMNS].[CHARACTER_MAXIMUM_LENGTH], [INFORMATION_SCHEMA].[COLUMNS].[NUMERIC_PRECISION], [INFORMATION_SCHEMA].[COLUMNS].[NUMERIC_SCALE], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_DEFAULT], [INFORMATION_SCHEMA].[COLUMNS].[COLLATION_NAME] 
FROM [INFORMATION_SCHEMA].[COLUMNS] 
WHERE [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2019-12-26 16:18:14,646 INFO sqlalchemy.engine.base.Engine ('Raw.Stage_Stock_info', 'dbo')
2019-12-26 16:18:14,710 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-26 16:18:14,713 INFO sqlalchemy.

2019-12-26 16:18:16,295 INFO sqlalchemy.engine.base.Engine (0, 'NYSE', 'Public Utilities', 'Power Generation', '94.00', '709233', '1596841', '29802386077', 21.14, '20.76', '4.24', '2.96', 'Nov 12 2019', 'Dec 16 2019', '3.301', -0.14, '94.97', '73.30', None, None, None, 'ED')
2019-12-26 16:18:16,353 INFO sqlalchemy.engine.base.Engine COMMIT
2019-12-26 16:18:16,376 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) ORDER BY [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME]
2019-12-26 16:18:16,377 INFO sqlalchemy.engine.base.Engine ('dbo', 'BASE TABLE')
2019-12-26 16:18:16,712 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA], [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_NAME], [INFORMATION_SCHEMA].[C

...

2019-12-26 16:21:13,204 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA], [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_NAME], [INFORMATION_SCHEMA].[COLUMNS].[IS_NULLABLE], [INFORMATION_SCHEMA].[COLUMNS].[DATA_TYPE], [INFORMATION_SCHEMA].[COLUMNS].[ORDINAL_POSITION], [INFORMATION_SCHEMA].[COLUMNS].[CHARACTER_MAXIMUM_LENGTH], [INFORMATION_SCHEMA].[COLUMNS].[NUMERIC_PRECISION], [INFORMATION_SCHEMA].[COLUMNS].[NUMERIC_SCALE], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_DEFAULT], [INFORMATION_SCHEMA].[COLUMNS].[COLLATION_NAME] 
FROM [INFORMATION_SCHEMA].[COLUMNS] 
WHERE [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2019-12-26 16:21:13,205 INFO sqlalchemy.engine.base.Engine ('Raw.Stage_Stock_info', 'dbo')
2019-12-26 16:21:13,265 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-26 16:21:13,268 INFO sqlalchemy.

2019-12-26 16:21:15,340 INFO sqlalchemy.engine.base.Engine (0, 'NYSE', 'Public Utilities', 'Natural Gas Distribution', '28.00', '4891283', '9488155', '28858882785', 183.15, '25.36', '0.13', '1.52', 'Dec 12 2019', 'Dec 30 2019', '6.3758', 0.67, '29.55', '20.36', None, None, None, 'WMB')
2019-12-26 16:21:15,401 INFO sqlalchemy.engine.base.Engine COMMIT
2019-12-26 16:21:15,426 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) ORDER BY [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME]
2019-12-26 16:21:15,426 INFO sqlalchemy.engine.base.Engine ('dbo', 'BASE TABLE')
2019-12-26 16:21:16,007 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA], [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_NAME], [INFORMATION

2019-12-26 16:21:17,864 INFO sqlalchemy.engine.base.Engine ('Raw.Stage_Stock_info', 'dbo')
2019-12-26 16:21:17,922 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-26 16:21:17,925 INFO sqlalchemy.engine.base.Engine INSERT INTO [Raw.Stage_Stock_info] ([index], [Exchange], [Sector], [Industry], [OneYrTarget], [ShareVolume], [AverageVolume], [MarketCap], [PERatio], [ForwardPE1Yr], [EarningsPerShare], [AnnualizedDividend], [ExDividendDate], [DividendPaymentDate], [Yield], [Beta], [FiftTwo Week High], [FiftTwo Week Low], [SpecialDividendDate], [SpecialDividendAmount], [SpecialDividendPaymentDate], [Stock]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2019-12-26 16:21:17,926 INFO sqlalchemy.engine.base.Engine (0, 'NYSE', 'Technology', 'Computer peripheral equipment', '38.00', '758865', '2200011', '8102736022', 13.2, '11.22', '2.84', '1.00', 'Dec 30 2019', 'Jan 31 2020', '2.67589', 1.15, '39.47', '18.58', None, None, None, 'XRX')
2019-12-26 16:21:17,993

2019-12-26 16:21:20,829 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA], [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_NAME], [INFORMATION_SCHEMA].[COLUMNS].[IS_NULLABLE], [INFORMATION_SCHEMA].[COLUMNS].[DATA_TYPE], [INFORMATION_SCHEMA].[COLUMNS].[ORDINAL_POSITION], [INFORMATION_SCHEMA].[COLUMNS].[CHARACTER_MAXIMUM_LENGTH], [INFORMATION_SCHEMA].[COLUMNS].[NUMERIC_PRECISION], [INFORMATION_SCHEMA].[COLUMNS].[NUMERIC_SCALE], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_DEFAULT], [INFORMATION_SCHEMA].[COLUMNS].[COLLATION_NAME] 
FROM [INFORMATION_SCHEMA].[COLUMNS] 
WHERE [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2019-12-26 16:21:20,830 INFO sqlalchemy.engine.base.Engine ('Raw.Stage_Stock_info', 'dbo')
2019-12-26 16:21:20,888 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-26 16:21:20,890 INFO sqlalchemy.

2019-12-26 16:21:22,686 INFO sqlalchemy.engine.base.Engine (0, 'NYSE', 'Capital Goods', 'Fluid Controls', 'N/A', '25472', '21249', '38432619', 0.0, '99.87', '-6.74', 'N/A', 'N/A', 'N/A', '12.02', -0.91, '4.92', '2.79', None, None, None, 'AP')
2019-12-26 16:21:22,748 INFO sqlalchemy.engine.base.Engine COMMIT
2019-12-26 16:21:22,771 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) ORDER BY [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME]
2019-12-26 16:21:22,772 INFO sqlalchemy.engine.base.Engine ('dbo', 'BASE TABLE')
